In [10]:
%run ipy_setup.py

In [78]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sqlalchemy

In [4]:
#branded food DB
nutrients = pd.read_table("../data/USDA_BFPD_csv/Nutrients.csv",sep=",")
products = pd.read_table("../data/USDA_BFPD_csv/Products.csv",sep=",")
serving_sizes = pd.read_table("../data/USDA_BFPD_csv/Serving_size.csv",sep=",")
len(list(zip(nutrients["Nutrient_Code"],nutrients["Nutrient_name"])))

import collections
collections.Counter(list(zip(nutrients["Nutrient_Code"],nutrients["Nutrient_name"])))

/Users/ryanneff/anaconda3/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [73]:
columns_nut_data=[i.strip() for i in '''ndb_id 
    nutr_id 
    nutr_value 
    num_data_pts 
    std_err 
    src_code 
    deriv_code 
    ref_ndb_no 
    add_nutr_mark 
    num_studies 
    min_val 
    max_val 
    deg_freedom 
    low_eb 
    up_eb 
    stat_cmt 
    addmod_date'''.split("\n")]

columns_food_desc = [i.strip() for i in """ndb_id 
    foodgroup_code 
    long_desc 
    short_desc 
    common_name 
    manufacturer_name 
    survey 
    ref_desc 
    ref_use 
    scientific_name 
    nit_factor 
    prot_factor 
    fat_factor 
    carb_factor""".split("\n")]

columns_nut_def = [i.strip() for i in '''nutr_id 
    units 
    tagname 
    nutr_desc 
    num_decimal_pts 
    sr_order'''.split("\n")]

columns_weight = [i.strip() for i in '''ndb_id 
    seq 
    amount 
    measure_desc 
    gm_weight 
    num_data_pts 
    std_dev'''.split("\n")]

nut_data = pd.read_table("../data/USDA_SR-Leg_csv/NUT_DATA.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_nut_data,encoding='latin-1')

food_desc = pd.read_table("../data/USDA_SR-Leg_csv/FOOD_DES.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_food_desc,encoding='latin-1')

nut_def = pd.read_table("../data/USDA_SR-Leg_csv/NUTR_DEF.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_nut_def,encoding='latin-1')

food_meas = pd.read_table("../data/USDA_SR-Leg_csv/WEIGHT.txt",dtype=str,
                         sep="^",quotechar="~",header=None,
                         names=columns_weight,encoding='latin-1')

#nut_data.set_index("NDB_No",inplace=True)
food_desc.set_index("ndb_id",inplace=True)
#food_meas.set_index("NDB_No",inplace=True)

nut_def.set_index("nutr_id",inplace=True)

In [72]:
food_desc[["steak" in i for i in food_desc["long_desc"]]]

,foodgroup_code,long_desc,short_desc,common_name,manufacturer_name,survey,ref_desc,ref_use,scientific_name,nit_factor,prot_factor,fat_factor,carb_factor
ndb_id,,,,,,,,,,,,,
10080,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STEAKS),L...","URMIS #3186, Pork shoulder chop, Shoulder blad...",NaN,Y,"Bone 9%, Connective tissue 15%",24,NaN,6.25,4.27,9.02,3.87
10081,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STKS),LN ...","URMIS #3186, Pork shoulder chop, Shoulder blad...",NaN,Y,"Bone 9%, Connective tissue 17%",26,NaN,6.25,4.27,9.02,3.87
10082,1000,"Pork, fresh, shoulder, blade, boston (steaks),...","PORK,FRSH,SHLDR,BLADE,BOSTON (STEAKS),LN&FAT,C...",NaN,NaN,Y,Bone,23,NaN,6.25,4.27,9.02,3.87
10084,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STEAKS),L...","URMIS # 3186, Pork shoulder chop, Shoulder bla...",NaN,Y,"Bone, 9%, Connective tissue, 15%, Separable fa...",32,NaN,6.25,4.27,9.02,3.87
10085,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BOSTON BUTT),BLADE (STEAKS),L...","URMIS #3186, Pork shoulder chop, Shoulder blad...",NaN,Y,"Bone, 9%, Connective tissue, 17%, Separable fa...",32,NaN,6.25,4.27,9.02,3.87
10086,1000,"Pork, fresh, shoulder, blade, boston (steaks),...","PORK,FRSH,SHLDR,BLADE,BOSTON (STEAKS),LN,CKD,BRLD",NaN,NaN,Y,"Bone 23%, separable fat 10%",33,NaN,6.25,4.27,9.02,3.87
10149,1000,"Pork, cured, ham, steak, boneless, extra lean,...","PORK,CURED,HAM,STEAK,BNLESS,EX LN,UNHTD",NaN,NaN,NaN,NaN,0,NaN,6.25,4.27,9.02,3.87
10945,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BSTN BTT),BLDE (STKS),LN,W/AD...",NaN,NaN,Y,"Bone, 9%, Connective tissue, 19%, Separable fa...",35,NaN,NaN,4.27,9.02,3.87
10946,1000,"Pork, fresh, shoulder, (Boston butt), blade (s...","PORK,FRSH,SHLDR,(BSTN BUTT),BLDE (STKS),LN,W/ ...",NaN,NaN,Y,"Bone, 7%, Connective tissue, 16%, Separable fa...",32,NaN,NaN,NaN,NaN,NaN


In [88]:
nut_def

,units,tagname,nutr_desc,num_decimal_pts,sr_order
nutr_id,,,,,
203,g,PROCNT,Protein,2,600
204,g,FAT,Total lipid (fat),2,800
205,g,CHOCDF,"Carbohydrate, by difference",2,1100
207,g,ASH,Ash,2,1000
208,kcal,ENERC_KCAL,Energy,0,300
209,g,STARCH,Starch,2,2200
210,g,SUCS,Sucrose,2,1600
211,g,GLUS,Glucose (dextrose),2,1700
212,g,FRUS,Fructose,2,1800


In [82]:
nut_data["nutr_value"] = nut_data["nutr_value"].astype(float)

In [86]:
{a:b for a,b in zip(nut_def.index,nut_def.nutr_desc)}

{'203': 'Protein',
 '204': 'Total lipid (fat)',
 '205': 'Carbohydrate, by difference',
 '207': 'Ash',
 '208': 'Energy',
 '209': 'Starch',
 '210': 'Sucrose',
 '211': 'Glucose (dextrose)',
 '212': 'Fructose',
 '213': 'Lactose',
 '214': 'Maltose',
 '221': 'Alcohol, ethyl',
 '255': 'Water',
 '262': 'Caffeine',
 '263': 'Theobromine',
 '268': 'Energy',
 '269': 'Sugars, total',
 '287': 'Galactose',
 '291': 'Fiber, total dietary',
 '301': 'Calcium, Ca',
 '303': 'Iron, Fe',
 '304': 'Magnesium, Mg',
 '305': 'Phosphorus, P',
 '306': 'Potassium, K',
 '307': 'Sodium, Na',
 '309': 'Zinc, Zn',
 '312': 'Copper, Cu',
 '313': 'Fluoride, F',
 '315': 'Manganese, Mn',
 '317': 'Selenium, Se',
 '318': 'Vitamin A, IU',
 '319': 'Retinol',
 '320': 'Vitamin A, RAE',
 '321': 'Carotene, beta',
 '322': 'Carotene, alpha',
 '323': 'Vitamin E (alpha-tocopherol)',
 '324': 'Vitamin D',
 '325': 'Vitamin D2 (ergocalciferol)',
 '326': 'Vitamin D3 (cholecalciferol)',
 '328': 'Vitamin D (D2 + D3)',
 '334': 'Cryptoxanthin, be

In [94]:
a = pd.merge(food_desc,nut_data.pivot_table(values="nutr_value",index="ndb_id",columns="nutr_id"),left_index=True,right_index=True)
b = a.rename(columns={a:b+" ("+c+")" for a,b,c in zip(nut_def.index,nut_def.nutr_desc,nut_def.units)})

In [96]:
b.reset_index(inplace=True)
b.to_csv("foods_with_all_data.tsv",sep="\t")

In [97]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../medtracker/food.db', echo=False)

In [76]:
nut_data.to_sql("nut_data",con=engine)
nut_def.to_sql("nut_def",con=engine)
food_desc.to_sql("food_desc",con=engine)
food_meas.to_sql("food_meas",con=engine)

In [107]:
a = a.rename(columns={a:"nutr_"+a for a in nut_def.index})

In [110]:
a.to_sql("food_desc_full",con=engine,if_exists="replace")

In [104]:
type(a["203"][0])

numpy.float64

In [109]:
for col in a.columns:
    type_col = str(type(a[col][0]))
    if "str" in type_col:
        type_col = "db.Text"
    if "float" in type_col:
        type_col = "db.Double"
    else:
        type_col = "db.Text"
    print(col + " = db.Column("+type_col+")")

foodgroup_code = db.Column(db.Text)
long_desc = db.Column(db.Text)
short_desc = db.Column(db.Text)
common_name = db.Column(db.Double)
manufacturer_name = db.Column(db.Double)
survey = db.Column(db.Text)
ref_desc = db.Column(db.Double)
ref_use = db.Column(db.Text)
scientific_name = db.Column(db.Double)
nit_factor = db.Column(db.Text)
prot_factor = db.Column(db.Text)
fat_factor = db.Column(db.Text)
carb_factor = db.Column(db.Text)
nutr_203 = db.Column(db.Double)
nutr_204 = db.Column(db.Double)
nutr_205 = db.Column(db.Double)
nutr_207 = db.Column(db.Double)
nutr_208 = db.Column(db.Double)
nutr_209 = db.Column(db.Double)
nutr_210 = db.Column(db.Double)
nutr_211 = db.Column(db.Double)
nutr_212 = db.Column(db.Double)
nutr_213 = db.Column(db.Double)
nutr_214 = db.Column(db.Double)
nutr_221 = db.Column(db.Double)
nutr_255 = db.Column(db.Double)
nutr_262 = db.Column(db.Double)
nutr_263 = db.Column(db.Double)
nutr_268 = db.Column(db.Double)
nutr_269 = db.Column(db.Double)
nutr_287 = db.Column(db